## School project - 5MLRE
The following notebook was created for a school project to create an anime recommendation system. The subject and the questions are available in the appendix.

The group members who participated in this project are:
- AMIMI Lamine
- BEZIN Théo
- LECOMTE Alexis
- PAWLOWSKI Maxence

### Main index
1. Data analysis
2. **Collaborative filtering (you are here)**
3. Content-based filtering
4. _Appendix_

# 2 - Collaborative filtering
In the previous notebook, we loaded, cleaned and studied the [MyAnimeList](https://myanimelist.net/) datasets. Now that we know them better, we will start to create the recommendation system using collaborative filtering. Collaborative filtering is a technique that filters out items that a user might like based on feedback from similar users. There are two sub-techniques: User-based collaborative filtering and article-based collaborative filtering.

### Index
<ol type="A">
  <li>Notebook initialization</li>
  <li>Collaborative filtering: unfiltered training</li>
  <li>Collaborative filtering: filtered training</li>
  <li>Getting the Top-N</li>
  <li>Conclusion of the collaborative filtering</li>
</ol>

## A - Notebook initialization
### A.1 - Imports

In [7]:
# OS and filesystem
import os
import sys
from pathlib import Path
import random

# Data
import pandas
from matplotlib import pyplot
import matplotx

# Model processing
import surprise

# Misc.
from ast import literal_eval

# Local files
sys.path.append(os.path.join(os.pardir, os.pardir))
import helpers

### A.2 - Package initialization

In [2]:
pyplot.rcParams.update(pyplot.rcParamsDefault)
pyplot.style.use(matplotx.styles.dracula)  # Set the matplotlib style

### A.3 - Constants

In [3]:
# Filesystem paths
PARENT_FOLDER = Path.cwd()
DATA_FOLDER = (PARENT_FOLDER / ".." / ".." / "data").resolve()
MODELS_FOLDER = (PARENT_FOLDER / ".." / ".." / "models").resolve()
TEMP_FOLDER = (PARENT_FOLDER / ".." / ".." / "temp").resolve()

# Plots
FIG_SIZE = (12, 7)

# Misc.
RANDOM_STATE = 2077

### A.4 - Datasets loading

In [4]:
data_reader = surprise.Reader(line_format="user item rating", sep=",", rating_scale=(-1, 10), skip_lines=1)
data = surprise.Dataset.load_from_file(file_path=(DATA_FOLDER / "rating2.csv"), reader=data_reader)

# Load a smaller sample of the dataset instead of the 8M rows
# data2 = pandas.read_csv((DATA_FOLDER / "rating.csv"), dtype={"user_id": str, "anime_id": str})
# data = data2[data2["rating"] >= 0.0]
# data = data.sample(n=500_000)

# data_reader = surprise.Reader(rating_scale=(1, 10))
# data = surprise.Dataset.load_from_df(df=data[["user_id", "anime_id", "rating"]], reader=data_reader)

In [5]:
data_anime = pandas.read_csv(DATA_FOLDER / "anime_cleaned.csv", converters={"genre_split": literal_eval})

In [6]:
rankings = pandas.Series(data=data_anime["rank_num_ratings"].values, index=data_anime["anime_id"]).to_dict()

In [7]:
evaluator = helpers.ml.ModelEvaluator(dataset=data, rankings=rankings, models_folder=MODELS_FOLDER, seed=RANDOM_STATE)

Constructing sets. This can take a while...
   > Building train/test sets...
   > Building LeaveOneOut sets...
   > Building full sets...
   > Preparing the similarities model...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.



# B - Collaborative filtering: unfiltered training
As we explained earlier, Collaborative filtering is a method used to make personalized recommendations by analyzing a user's past preferences or behaviors and comparing them to those of similar users. There are two sub-techniques: User-based collaborative filtering and article-based collaborative filtering.

- User-based: focuses on finding similar users that are looking like the target user in terms of preferences, liked items and user's navigation.
- Item-based: focuses on finding similar items based on the user's previous interactions with other items.

User-based is more relevant for entertainment-related items, as this approach would recommend items that other users with similar preferences have liked. There are a lot of parameters in terms of preference nuances. Item-based recommendations are more pertinent to online shops, which recommend products based on their characteristics. We are talking about individual tastes here.

In our case, user-based filtering should give better results. But in this notebook we will test our models with both methods.

## B.1 - Slope One
Slope One is a collaborative filtering algorithm designed for recommendations. Its lightweight and simple design calculates the average difference between the user's items rating and uses this information to predict the user's potential rating on an unseen article.

In [8]:
evaluator.run_model(name="Slope One", model=surprise.SlopeOne, hyper_params=None, measure_key="rmse", override=False)

Testing "Slope One".
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Building the top-N...
   > Fitting on the LOOCV...
   > Fitting on the full set...
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {}
RMSE: 2.187853
MAE: 1.397903

Hit rate: 0.402010%
Hit rate per rating value:
Rating	Hit rate
10.0	2.139037%
Cumulative hit rate (min_rating=3.0): 0.481928%
Average reciprocal hit rate: 0.0021775544388609714
User coverage (num_users=995, min_rating=3.0): 100.000000%
Diversity: 0.666667
Novelty: 3181.175146

Testing of the "Slope One" model successfully completed in 0:08:08.516542.
Grid search: N/A
Training and testing: 0:00:03.649915
Top-N building: 0:08:00.837321


## B.2 - KNN Basic
KNN Basic (K-Nearest Neighbors) is another collaborative filtering algorithm used for recommandation systems. It consists of finding the most similar "K" items or users based on a similarity metric. It then calculates the weighted average of the ratings of the items found to predict the user's rating for the target item or recommend items based on the user.

In [9]:
evaluator.run_model(
    name="KNN Basic",
    model=surprise.KNNBasic,
    hyper_params={
        "k": [20, 40, 60],
        "min_k": [1, 2, 3, 5],
        "sim_options": {
            "name": ["cosine", "msd", "pearson", "pearson_baseline"],
            "user_based": [True, False]
        }
    },
    measure_key="rmse",
    override=False
)

Testing "KNN Basic".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Building the top-N...
   > Fitting on the LOOCV...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
   > Fitting on the full set...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'k': 40, 'min_k': 1, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}}
RMSE: 2.220969
MAE: 1.376290

Hit rate: 0.402010%
Hit rate per rating value:
Rating	Hit rate
-1.0	0.617284%
8.0	0.943396%
10.0	0.534759%
Cumulative hit rate (min_rating=3.0): 0.361446%
Average reciprocal hit rate: 0.0020938023450586267
User coverage (num_users

## B.3 - KNN With Means
KNN With Means is a variant of the KNN Basic algorithm. This time, the algorithm adjusts the previously calculated weighted average by adding the overall average user or article rating.

In [10]:
evaluator.run_model(
    name="KNN With Means",
    model=surprise.KNNWithMeans,
    hyper_params={
        "k": [20, 40, 60],
        "min_k": [1, 2, 3, 5],
        "sim_options": {
            "name": ["cosine", "msd", "pearson", "pearson_baseline"],
            "user_based": [True, False]
        }
    },
    measure_key="rmse",
    override=False
)

Testing "KNN With Means".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Building the top-N...
   > Fitting on the LOOCV...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
   > Fitting on the full set...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'k': 40, 'min_k': 5, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}
RMSE: 2.131681
MAE: 1.324676

Hit rate: 2.613065%
Hit rate per rating value:
Rating	Hit rate
4.0	8.333333%
7.0	0.632911%
8.0	2.830189%
9.0	1.092896%
10.0	8.556150%
Cumulative hit rate (min_rating=3.0): 3.132530%
Average reciprocal hit rate: 0.0085574698891

## B.4 - KNN With Z-Score
KNN With Z-Score is another variant of the KNN algorithm that takes into account the average ratings and standard deviations of users or items for predictions. In addition to the previous steps, the algorithm calculates the Z-Score by subtracting the average score and dividing the result by the standard deviation on the weighted average. With this method, this algorithm normalizes the ratings by trends and variabilities, which means better accuracy for predictions and recommendations.

In [11]:
evaluator.run_model(
    name="KNN With Z-Score",
    model=surprise.KNNWithZScore,
    hyper_params={
        "k": [20, 40, 60],
        "min_k": [1, 2, 3, 5],
        "sim_options": {
            "name": ["cosine", "msd", "pearson", "pearson_baseline"],
            "user_based": [True, False]
        }
    },
    measure_key="rmse",
    override=False
)

Testing "KNN With Z-Score".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Building the top-N...
   > Fitting on the LOOCV...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
   > Fitting on the full set...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'k': 40, 'min_k': 5, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}
RMSE: 2.130252
MAE: 1.299102

Hit rate: 2.311558%
Hit rate per rating value:
Rating	Hit rate
7.0	0.632911%
8.0	0.943396%
9.0	1.639344%
10.0	9.090909%
Cumulative hit rate (min_rating=3.0): 2.771084%
Average reciprocal hit rate: 0.007682459918640822
User c

## B.5 - KNN Baseline
KNN Baseline is simpler than the previous algorithm. It calculates the distance between the raw values of the features that we want to use for our prediction. The counterpart of this method is the loss of accuracy depending to the scales or ranges of the features.

In [12]:
evaluator.run_model(
    name="KNN Baseline",
    model=surprise.KNNBaseline,
    hyper_params={
        "k": [20, 40, 60],
        "min_k": [1, 2, 3, 5],
        "sim_options": {
            "name": ["cosine", "msd", "pearson", "pearson_baseline"],
            "user_based": [True, False]
        },
        "bsl_options": {
            "method": ["als"],
            "n_epochs": [5, 10, 15],
        }
    },
    measure_key="rmse",
    override=False
)

Testing "KNN Baseline".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Building the top-N...
   > Fitting on the LOOCV...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
   > Fitting on the full set...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'k': 20, 'min_k': 3, 'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'bsl_options': {'method': 'als', 'n_epochs': 15}}
RMSE: 2.021351
MAE: 1.246453

Hit rate: 2.814070%
Hit rate per rating value:
Rating	Hit rate
-1.0	0.617284%
7.0	1.265823%
8.0	0.943396%
9.0	3.825137%
10.0	8.556150%
Cumulative hit rate (min_rating=3.0): 3.25

## B.6 - Non-negative Matrix Factorization
Non-negative Matrix Factorization (NMF) is a technique used to facilitate the interpretation of non-negative matrices*¹* of data. For this, the algorithm tries to find a way to represent a non-negative matrix in smaller non-negative matrices. It can then better interpret the data structures and its predictions are improved.

*1: A non-negative matrix is a matrix where all elements are greater than or equal to zero.*

In [13]:
evaluator.run_model(
    name="Non-negative Matrix Factorization",
    model=surprise.NMF,
    hyper_params={
        "n_factors": [5, 15, 25],
        "n_epochs": [25, 50, 75],
        "biased": [True, False]
    },
    measure_key="rmse",
    override=False
)

Testing "Non-negative Matrix Factorization".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Building the top-N...
   > Fitting on the LOOCV...
   > Fitting on the full set...
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'n_factors': 5, 'n_epochs': 25, 'biased': True}
RMSE: 2.542902
MAE: 1.641667

Hit rate: 0.301508%
Hit rate per rating value:
Rating	Hit rate
10.0	1.604278%
Cumulative hit rate (min_rating=3.0): 0.361446%
Average reciprocal hit rate: 0.0018425460636515912
User coverage (num_users=995, min_rating=3.0): 100.000000%
Diversity: 0.127392
Novelty: 882.962780

Testing of the "Non-negative Matrix Factorization" model successfully completed in 0:06:50.143506.
Grid search: 0:05:08.462348
Training and testing: 0:00:00.713953
Top-N building: 0:01:36.056400


## B.7 - Co-clustering
The goal of Co-clustering is to find a way to group similar rows and columns of a matrix, like patterns, to make them more apparent. This method is useful when working on datasets with complex row and column relationships. The algorithm will first find a correlation between the rows and columns of the dataset. It will then use the k-mean method to group the data before interpreting the results.

In [14]:
evaluator.run_model(
    name="Co-clustering",
    model=surprise.CoClustering,
    hyper_params={
        "n_cltr_u": [1, 3, 5],
        "n_cltr_i": [1, 3, 5],
        "n_epochs": [10, 20, 30],
    },
    measure_key="rmse",
    override=False
)

Testing "Co-clustering".
Running GridSearchCV...
Computing metrics...
Calculating the accuracy (RMSE, MAE)...
Building the top-N...
   > Fitting on the LOOCV...
   > Fitting on the full set...
Built top-N for each user (n=10, min_rating=3.0)
Built top-N for each user (n=10, min_rating=3.0)

Best params: {'n_cltr_u': 1, 'n_cltr_i': 1, 'n_epochs': 10}
RMSE: 2.295664
MAE: 1.489803

Hit rate: 2.010050%
Hit rate per rating value:
Rating	Hit rate
6.0	1.851852%
7.0	0.632911%
8.0	1.415094%
9.0	2.185792%
10.0	5.882353%
Cumulative hit rate (min_rating=3.0): 2.409639%
Average reciprocal hit rate: 0.005360134003350083
User coverage (num_users=995, min_rating=3.0): 93.165829%
Diversity: 0.977778
Novelty: 2220.936524

Testing of the "Co-clustering" model successfully completed in 0:12:24.688897.
Grid search: 0:11:08.855068
Training and testing: 0:00:00.930792
Top-N building: 0:01:10.142236


## B.8 - Comparing performance
Before we compare all the previous results, let's define a few important terms that we need to understand to properly compare the performance of our models.

<u>Machine Learning metrics:</u>
- Root Mean Squared Error (RMSE): measure of the average deviation of the predicted values of the model. The lower, the better.
- Mean Absolute Error (MAE): refers to the magnitude of difference between the prediction of an observation and the true value of that observation.

<u>Recommendation systems metrics:</u>
- Hit rate: the proportion of recommended items that are relevant to the user, expressed in percent.
- Hit rate per rating value: is the hit rate but calculated independently for each of the possible ratings (from one to ten in our case).
- Cumulative hit rate: is also the hit rate calculated for all rating values up to a certain threshold.
- Average reciprocal hit rank (ARHR): is the average of the reciprocal ranks of the relevant items in the recommended list.
- User coverage: is the proportion of users for whom the system is able to make recommendations.
- Diversity: is the variety or dissimilarity of items recommended to users.
- Novelty: is the degree to which recommended items are new or unexpected to the user.

We can now compare our models. We are going to observe which is the best model for each metrics, and then conclude on the overall best model.

The model with the most interesting metric values is the KNN Baseline due to its good performance on RMSE, MAE and hit rate. It has a reasonable training time of seven seconds. Even though the grid search takes three hours, we already have the best parameters for this model, so we don't need to run the grid search again. In another hand, KNN With Means show us an interesting performance for ARHR and a training of three seconds. KNN Basic is automatically excluded because 35% of its predictions were impossible (one of the parameters was unknown to it).

Overall, based on the metrics and results we have at this moment of the training, KNN Baseline appears to be the most efficient model.

## D - Getting the Top-N
The final step is to display the top-N of a user. We start by loading the previously saved top of our best model.

In [11]:
top_n_knn_baseline = helpers.ml.Model.load_top_n(filepath=(MODELS_FOLDER / "KNN_Baseline__topN-full.pkl"))

We then pick a random user from our dataset.

In [8]:
random_user_id = int(random.choice(list(set([r[0] for r in data.raw_ratings]))))
random_user_id

263

We define a function that build a human-readable table from the top-N.

In [17]:
def get_top_n_of(user_id: int, top_n: dict[int, list], items_df: pandas.DataFrame, auto_print: bool = False) -> pandas.DataFrame:
    """ Returns the formatted top-N recommendation for a specific user. """
    user_top = []

    for top_item_id, estimated_rating, _ in top_n[user_id]:
        item = items_df[items_df["anime_id"] == top_item_id].iloc[0]
        user_top.append({
            "Name": item["name"],
            "Genre": item["genre"],
            "Num. ratings": item["num_ratings"],
            "Mean ratings": item["rating"],
            "User estimated rating": estimated_rating
        })

    return pandas.DataFrame(data=user_top)

In [18]:
get_top_n_of(user_id=random_user_id, top_n=top_n_knn_baseline, items_df=data_anime, auto_print=False)

,Name,Genre,Num. ratings,Mean ratings,User estimated rating
0,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",21494,9.26,9.781889
1,Hajime no Ippo,"Comedy, Drama, Shounen, Sports",4273,8.83,9.670955
2,Rainbow: Nisha Rokubou no Shichinin,"Drama, Historical, Seinen, Thriller",2716,8.64,9.654531
3,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",7477,9.13,9.647590
4,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",15518,9.06,9.625787
5,Monster,"Drama, Horror, Mystery, Police, Psychological,...",4079,8.72,9.567637
6,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",2126,9.11,9.521137
7,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",16955,8.78,9.482833
8,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",8271,8.92,9.426137
9,Clannad,"Comedy, Drama, Romance, School, Slice of Life,...",18746,8.30,9.384167


Here is the top-10 of the randomly selected user.

## E - Conclusion of the collaborative filtering
**TODO: Add text**